# Predicting Count of IG Members in DNA_Test_Data_v3

### Notebook automatically generated from your model

Model Ridge (L2) regression, trained on 2020-07-27 06:52:21.

#### Generated on 2020-07-27 06:54:36.055218

prediction
This notebook will reproduce the steps for a REGRESSION on  DNA_Test_Data_v3.
The main objective is to predict the variable Count of IG Members

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [6]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [7]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [8]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = [{u'metaType': u'PROCESSOR', u'alwaysShowComment': False, u'disabled': False, u'params': {u'clear': False, u'lowerBound': u'1.000', u'upperBound': u'17.00', u'columns': [u'Count of IG Members']}, u'preview': False, u'type': u'MinMaxProcessor'}]
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'bigint', u'name': u'Sign up Date_year'}, {u'type': u'bigint', u'name': u'Sign up Date_month'}, {u'type': u'date', u'name': u'Sign up Date'}, {u'type': u'string', u'name': u'CC'}, {u'type': u'string', u'name': u'Category'}, {u'type': u'bigint', u'name': u'Count of IG Signups'}, {u'type': u'double', u'name': u'Count of IG Members'}, {u'type': u'double', u'name': u'Signup Rate'}]}

ml_dataset_handle = dataiku.Dataset('DNA_Test_Data_v3')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 6.98 ms, sys: 2.68 ms, total: 9.66 ms
Wall time: 18.7 ms
Base data has 110 rows and 8 columns


,Sign up Date_year,Sign up Date_month,Sign up Date,CC,Category,Count of IG Signups,Count of IG Members,Signup Rate
0,2010,1,2010-01-01,CC11,Personal & Career Skills,1.0,1.0,1.0
1,2010,3,2010-01-03,CC11,Personal & Career Skills,1.0,1.0,1.0
2,2010,4,2010-01-04,CC11,Personal & Career Skills,1.0,1.0,1.0
3,2010,5,2010-01-05,CC11,Personal & Career Skills,4.0,2.0,2.0
4,2010,6,2010-01-06,CC11,Personal & Career Skills,1.0,1.0,1.0


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [9]:
ml_dataset = ml_dataset[[u'Category', u'Sign up Date_month', u'Sign up Date_year', u'Count of IG Members']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [10]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = [u'Category']
numerical_features = [u'Sign up Date_month', u'Sign up Date_year']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]'):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We renamed the target variable to a column named target

In [11]:
ml_dataset['__target__'] = ml_dataset['Count of IG Members']
del ml_dataset['Count of IG Members']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

Important note: your model used a more advanced cross-validation strategy.
For the purpose of this notebook, it has been simplified to a random split of
a single dataset

In [12]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

Train data has 88 rows and 4 columns
Test data has 22 rows and 4 columns


#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [13]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u'Sign up Date_month'}, {'impute_with': u'MEAN', 'feature': u'Sign up Date_year'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

Imputed missing values in feature Sign up Date_month with value 6.38636363636
Imputed missing values in feature Sign up Date_year with value 2015.30681818


/home/ec2-user/dataiku/dataiku-dss-7.0.2/python.packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [14]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = [u'Category']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Dummy-encoded feature Category
Dummy-encoded feature Category


/home/ec2-user/dataiku/dataiku-dss-7.0.2/python.packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Let's rescale numerical features

In [15]:
rescale_features = {u'Sign up Date_year': u'AVGSTD', u'Sign up Date_month': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

Rescaled Sign up Date_year
Rescaled Sign up Date_month


/home/ec2-user/dataiku/dataiku-dss-7.0.2/python.packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [16]:
train_X = train.drop('__target__', axis=1)
test_X = test.drop('__target__', axis=1)

train_Y = np.array(train['__target__'])
test_Y = np.array(test['__target__'])

Now we can finally create our model !

In [17]:
from sklearn.linear_model import RidgeCV
clf = RidgeCV(fit_intercept=True, normalize=True)

... And train it

In [18]:
%time clf.fit(train_X, train_Y)

CPU times: user 3.74 ms, sys: 0 ns, total: 3.74 ms
Wall time: 9.42 ms


RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=True, scoring=None, store_cv_values=False)

Build up our result dataset

In [19]:
%time _predictions = clf.predict(test_X)
predictions = pd.Series(data=_predictions, index=test_X.index, name='predicted_value')

# Build scored dataset
results_test = test_X.join(predictions, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'Count of IG Members'})

CPU times: user 1.25 ms, sys: 0 ns, total: 1.25 ms
Wall time: 841 µs


#### Results

You can measure the model's accuracy:

In [20]:
c =  results_test[['predicted_value', 'Count of IG Members']].corr()
print ('Pearson correlation: %s' % c['predicted_value'][1])

Pearson correlation: 0.3697243861977895


That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
